<a href="https://colab.research.google.com/github/Yinlipp/SoccerTrackChallenge-2025_Colab/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This is a baseline code without fine-tuning for the SoccerTrack Challenge 2025 (STC: https://sites.google.com/g.sp.m.is.nagoya-u.ac.jp/stc2025).
# The code is divided into two parts: (Participants may ignore the second part, only the MOT-format .txt tracking results from the first part need to be submitted.)
# 1.Using the official Deep-EIoU code (https://github.com/hsiangwei0903/Deep-EIoU) to perform tracking on videos and obtain tracking results in MOT-format .txt files.
# 2.Using the official HOTA code (https://github.com/JonathonLuiten/TrackEval)to evaluate the tracking results from step 1 with the ground truth using the HOTA metric.

# Cell 1 takes approximately 50 minutes to run on Colab with a T4 GPU.
# Install dependencies and run Deep-EIoU for video tracking inference.
!sudo apt-get install python3.7 python3.7-dev python3.7-distutils -y
!wget https://bootstrap.pypa.io/pip/3.7/get-pip.py
!python3.7 get-pip.py
!python3.7 -m pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 torchaudio==0.13.0 --index-url https://download.pytorch.org/whl/cu116
!python3.7 -m pip install cython_bbox gdown

# Deep-EIoU repository
!Git clone Deep-EIoU
!git clone https://github.com/hsiangwei0903/Deep-EIoU.git

# Reid requirement
%cd /content/Deep-EIoU/Deep-EIoU/reid
!python3.7 -m pip install -r requirements.txt

# main directory
%cd /content/Deep-EIoU/Deep-EIoU

# Download test video (STC dataset is available athttps://drive.google.com/drive/folders/1_o78gcL4j0xHxbRjSR1Evs4VLXCr2ncD),  detector and ReID model of Deep-EIoU
# You can replace the video here with your own or other STC videos.
!gdown --fuzzy 'https://drive.google.com/file/d/1DQj5-kiU4VySymDXZuV84-nU1AZ4HZaj/view?usp=drive_link' # STC Training dataset video 117092.mp4
!gdown --fuzzy 'https://drive.google.com/file/d/1834kh10-X0Tu743fgmN7jXPVDKgq4ZqR/view?usp=drive_link' --output checkpoints/best_ckpt.pth.tar  # detector
!gdown --fuzzy 'https://drive.google.com/file/d/14zzlm1nI9Ws_Il9RYNChwPC7Fsul7xwl/view?usp=drive_link' --output checkpoints/sports_model.pth.tar-60 #reid

# Inferevce
!python3.7 -m pip install https://github.com/KaiyangZhou/deep-person-reid/archive/master.zip  # torchreid
!python3.7 tools/demo.py --path 117092.mp4 --save_result True
# The video ouput is located at /content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/(timestamp)/117092.mp4
# The tracking result .txt output is located at:/content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/*.txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7 libpython3.7-dev libpython3.7-minimal libpython3.7-stdlib
  python3.7-lib2to3 python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7 libpython3.7-dev libpython3.7-minimal libpython3.7-stdlib
  python3.7 python3.7-dev python3.7-distutils python3.7-lib2to3
  python3.7-minimal
0 upgraded, 9 newly installed, 0 to remove and 30 not upgraded.
Need to get 11.1 MB of archives.
After this operation, 41.8 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minimal amd64 3.7.17-1+jammy1 [1,837 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubunt

In [2]:
# Cell 12 computes the HOTA metric using the tracking results from cell 1 and the STC dataset's ground truth.
# Official  HOTA repository
%cd /content/
!git clone https://github.com/JonathonLuiten/TrackEval.git
!python3.7 -m pip install lap cython motmetrics

%cd /content/TrackEval/
!python3.7 -m pip install -r requirements.txt

# To compute the HOTA metric, the GT files (Ground Truth) and prediction files (tracker outputs) must be placed in the format and directory structure required by TrackEval.
!mkdir -p /content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt

# Download the GT files and place them in the required directory structure.
# The GT file is placed in /content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt/gt.txt
# The tracking result file is placed in /content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data/117092.txt
!gdown --fuzzy 'https://drive.google.com/file/d/1cQeC4c0FG8i8Xayw3LdzhEU5p42s7zD1/view?usp=drive_link' --output /content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt/gt.txt

# Pred file
!mkdir -p /content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data
import glob, shutil,os
files_path = glob.glob('/content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/*.txt', recursive=True)

# Copy the Deep-EIoU inference results into the required directory structure.
for file in files_path:
    shutil.copy(file, os.path.join("/content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data", '117092.txt'))

# To mimic the MOT17 dataset structure
# seqmap text
seqmap_text = "name\n117092"
os.makedirs("/content/TrackEval/data/gt/mot_challenge/mydataset/seqmaps/", exist_ok=True)
# MOT17-test.txt
with open("/content/TrackEval/data/gt/mot_challenge/mydataset/seqmaps//mydataset-test.txt", "w") as f:
    f.write(seqmap_text)
# print("✅seqmap (MOT17-test.txt):\n", seqmap_text)
# seqinfo.ini
ini_content = (
    "[Sequence]\n"
    "name=117092\n"
    "imDir=img1\n"
    "frameRate=30\n"
    "seqLength=1000000\n"
    "imWidth=1920\n"
    "imHeight=1080\n"
    "imExt=.jpg"
)
# seqinfo.ini
os.makedirs("/content/TrackEval/data/gt/mot_challenge/mydataset/117092", exist_ok=True)
with open("/content/TrackEval/data/gt/mot_challenge/mydataset/117092/seqinfo.ini", "w") as f:
    f.write(ini_content)
# print("✅ ini_content)

# The frame indices in both the GT and prediction result files must start from 1.
def update_file_in_place(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    with open(file_path, 'w', encoding='utf-8') as f:
        for line in lines:
            parts = line.strip().split(',')
            if parts:
                # line[0] + 1
                if parts[0].isdigit():
                    parts[0] = str(int(parts[0]) + 1)

                if len(parts) >= 3:
                    parts[-3] = '1'
                # ，2 space
                new_line = ' '.join(parts)
                f.write(new_line + '\n')
            else:
                f.write(line)
# Rewrite the .txt files to ensure they meet the required format.
gt_txt = '/content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt/gt.txt'
pred_path = '/content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data/117092.txt'
update_file_in_place(gt_txt)
update_file_in_place(pred_path)

# HOTA
%cd /content/TrackEval
!python3.7 scripts/run_mot_challenge.py \
--BENCHMARK mydataset \
--SPLIT_TO_EVAL test\
--GT_FOLDER data/gt/mot_challenge/mydataset \
--TRACKERS_FOLDER data/trackers/mot_challenge/mydataset \
--TRACKERS_TO_EVAL my_tracker \
--METRICS HOTA CLEAR Identity \
--SKIP_SPLIT_FOL True


/content
Cloning into 'TrackEval'...
remote: Enumerating objects: 924, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 924 (delta 340), reused 315 (delta 315), pack-reused 550 (from 1)
Receiving objects: 100% (924/924), 393.80 KiB | 11.93 MiB/s, done.
Resolving deltas: 100% (619/619), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 36.7 MB/s eta 0:00:00
/content/TrackEval
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of imageio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of imageio to determine which version is compatible with other requ